In [18]:
import os
import numpy as np
# 这个文件的目的是对solar数据集进行预处理

solar_raw_path = "/data/hanzhi/solar_AL.txt"
lines = open(solar_raw_path, 'r').readlines()

# note 先把原始数据读取出来
# (count_timestamps, sensors)
data = []
for line in lines:
    readings = line.split(",")
    reading = [float(i) for i in readings]
    data.append(reading)

data = np.array(data, dtype=np.float32)
print(data.shape, data[50], np.mean(data), np.std(data), np.min(data), np.max(data))

# note 再增加time of day的嵌入
tod = np.arange(144, dtype=np.float32)
tod /= 144
# (144) -> (ts) -> (ts, 1) -> (ts, sensors)
tod = np.tile(tod, data.shape[0]//144)[:, np.newaxis]
tod = np.tile(tod, (1, data.shape[1]))
# (ts, sensors, 2)
final_data = np.stack([data, tod], axis=-1)
# note 再保存到磁盘上
output_dir = "/data3/hanzhi/BasicTS/datasets/Solar"
os.makedirs(output_dir, exist_ok=True)
written_data = np.memmap(os.path.join(output_dir, "data.dat"), mode="write", shape=final_data.shape, dtype=final_data.dtype)
written_data[:] = final_data
written_data.flush()

(52560, 137) [ 5.1   6.9   7.45  4.35  0.65 19.35  8.55  7.55  2.4   7.9   4.6   1.6
  1.6   7.45  4.65  8.15  8.3   2.75  4.35  5.5   4.85  9.1  13.    0.7
  0.4   4.3   6.9   4.2   9.    1.2   5.4   0.8   0.6   1.65  4.15  7.5
  1.45  4.65  8.05  8.   27.55  3.4   9.2   5.6  16.6   6.    5.65  6.65
  4.75  8.3   5.1   5.25  0.25  4.75  6.55  6.3   5.85  8.35  7.    6.05
  2.75  8.    4.8   4.3   9.8   5.9   8.4   5.   21.45  4.65  0.3   5.95
  6.85  0.45  0.5   5.8   2.8   2.95  5.6   7.45  5.55  7.85  0.    7.25
  5.9   6.1   2.5   6.25  6.    6.05  4.8   8.4   6.    6.75  2.9   5.05
  2.15  3.3   7.6   4.25  6.25  6.25  7.05  6.5   6.4   0.4   7.25  4.35
  4.3   5.95  9.3   2.6   3.5   4.    0.15  0.    4.65  5.9   7.95  5.9
  1.75 24.6   0.55  3.55 16.7   0.2   1.55  4.6   4.65  1.15  0.35  8.3
  7.15  6.1   5.9  12.05 12.85] 6.3526506 10.15121 0.0 88.9
